# Data loading

In [1]:

import json
import requests
import time

import pandas as pd
import joblib


### Query Search

In [2]:
cities = [
    "Lisboa",
    "Porto",
    "Setúbal",
    "Braga",
    "Aveiro",
    "Faro",
    "Leiria",
    "Santarém",
    "Coimbra",
    "Viseu",
    "Viana do Castelo",
    "Vila Real",
    "Castelo Branco",
    "Évora",
    "Beja",
    "Guarda",
    "Bragança",
    "Portalegre",
]

### Exemplo

In [52]:
query = 'Porto'
websites = 'observador.pt'
max_items = 2000
from_ = '2019'
to_ = '2021'
link = f"https://arquivo.pt/textsearch?q={query}&siteSearch={websites}&maxItems={max_items}&dedupValue=1&prettyPrint=true&from={from_}&to={to_}"
f = requests.get(link)
df = json.loads(f.text)


In [54]:
len(df['response_items'])

2000

## Publico API

In [43]:
keyword= 'viana do castelo'
starting_date = '01-01-2010'
ending_date = '31-01-2010'
page_number = 0
link = f"https://www.publico.pt/api/list/search/?query={keyword}&start={starting_date}&end={ending_date}&page={page_number}"
f = requests.get(link)
df = json.loads(f.text)

In [44]:
len(df)

2

In [45]:
[f['fullUrl'] for f in df]

['https://www.publico.pt/2010/01/10/sociedade/noticia/frio-e-neve-deixam-norte-e-centro-do-pais-em-alerta-e-condicionam-transito-1417117',
 'https://www.publico.pt/2010/01/30/jornal/viana-elimina-passagens--de-nivel-18696347']

In [42]:
[f['fullUrl'] for f in df]

['https://www.publico.pt/2010/01/10/sociedade/noticia/frio-e-neve-deixam-norte-e-centro-do-pais-em-alerta-e-condicionam-transito-1417117',
 'https://www.publico.pt/2010/01/29/desporto/noticia/tribunal-emitiu-ordem-de-apreensao-de-autocarro-do-sp-braga-com-caracter-de-urgencia-1420490']

## Extração capitais de distrito

In [3]:
newspapers = ['expresso.pt' ,'publico.pt', 'jn.pt', 'dn.pt', 'cmjornal.pt', 'sol.sapo.pt', 'visao.sapo.pt', 'jornaldenegocios.pt', 'observador.pt']
others = ['turismodeportugal.pt', 'nit.pt']
sources = newspapers + others

In [4]:

max_items = 2000
from_ = 1996
to_ = 2023

columns=['city', 'title', 'content', 'year', 'tstamp', 'link']

data = pd.DataFrame(columns=columns)
for city in cities:
    for year in range(from_, to_):
        
        link = f"https://arquivo.pt/textsearch?q={city}&siteSearch={','.join(sources)} \
            &maxItems={max_items}&dedupValue=1&prettyPrint=true&from={str(year)}&to={str(year+1)}"
        
        try:
        
            r = requests.get(link)
            payload = json.loads(r.text)['response_items']
        
            for idx, article in enumerate(payload):
                
                time.sleep(1)
    
                data = pd.concat([data, pd.DataFrame({
                    'city': city, 'title': article['title'], 'content': requests.get(article['linkToExtractedText']).text, 
                    'year': year, 'tstamp': article['tstamp'], 'link': article['linkToArchive']}, index=[idx])
                ], ignore_index=True)
        
        except Exception as e:
            #print(e)
            data = pd.concat([data, pd.DataFrame({
                    'city': city, 'title': None, 'content': None, 
                    'year': year, 'tstamp': None, 'link': None
                    }, index=[idx])
            ], ignore_index=True)

data.to_csv('../data/scraping_data.csv.gz', compression='gzip')   

In [9]:
data.head()

,city,title,content,year,tstamp,link
0,Lisboa,POL | Local Lisboa,POL | Local Lisboa SECÇÕES 1ª Página Destaque ...,1999,19991111042737,https://arquivo.pt/wayback/19991111042737/http...
1,Lisboa,JN Editorial - Text57,JN Editorial - Text57 26 milhões para dar casa...,1999,19990822002536,https://arquivo.pt/wayback/19990822002536/http...
2,Lisboa,Outras Paginas,Outras Paginas 11 de Novembro de 1999 Igreja d...,1999,19991117215651,https://arquivo.pt/wayback/19991117215651/http...
3,Lisboa,JN Editorial - Texult1,JN Editorial - Texult1 Macau: Rão Kyao é o aut...,1999,19991118004529,https://arquivo.pt/wayback/19991118004529/http...
4,Lisboa,PÚBLICONLINE-Os Destaques da Primeira Página,PÚBLICONLINE-Os Destaques da Primeira Página S...,1999,19991012235908,https://arquivo.pt/wayback/19991012235908/http...


### Parallelism

In [4]:
def article_scrapper(city, sources):

    max_items = 10
    from_ = 1996
    to_ = 2023

    columns=['city', 'title', 'content', 'year', 'tstamp', 'link']
    data = pd.DataFrame(columns=columns)
    for year in range(from_, to_):
        
        link = f"https://arquivo.pt/textsearch?q={city}&siteSearch={','.join(sources)} \
            &maxItems={max_items}&dedupValue=1&prettyPrint=true&from={str(year)}&to={str(year+1)}"
        
        try:
        
            r = requests.get(link)
            payload = json.loads(r.text)['response_items']
        
            for idx, article in enumerate(payload):
                
                time.sleep(1)

                data = pd.concat([data, pd.DataFrame({
                    'city': city, 'title': article['title'], 'content': requests.get(article['linkToExtractedText']).text, 
                    'year': year, 'tstamp': article['tstamp'], 'link': article['linkToArchive']}, index=[idx])
                ], ignore_index=True)
        
        except Exception as e:
            
            data = pd.concat([data, pd.DataFrame({
                    'city': city, 'title': None, 'content': None, 
                    'year': year, 'tstamp': None, 'link': None
                    }, index=[idx])
            ], ignore_index=True)

    data.to_csv(f'../data/scraping_data.csv', index=False)   


In [5]:
joblib.Parallel(n_jobs=-1)(
        joblib.delayed(article_scrapper)(
            city=city,
            sources=sources,    
        )
        for city in cities
    )